In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from rdkit.Chem.Draw import IPythonConsole
from sklearn.metrics import r2_score as sklearn_r2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch.nn import BatchNorm1d, HuberLoss, L1Loss, Linear, ModuleList, MSELoss
from torch_geometric.nn import (
    GATv2Conv,
    GraphNorm,
    TopKPooling,
    global_max_pool,
    global_mean_pool,
)
from torcheval.metrics import R2Score
from torcheval.metrics.functional import r2_score
from twinning import twin


In [2]:
codes = pd.read_excel("codes.xlsx", engine="openpyxl", usecols=[0,1])

In [3]:
df = pd.read_excel("dataKP.xlsx", engine="openpyxl", usecols=[2,3,4])

In [4]:
codes.loc[len(df.index)] = ["hf2", "F[H][F-]"]
codes.dropna(inplace=True)

In [5]:
df["MP"] = df["MP"].astype(float)

In [6]:
df

,cation,anion,MP
0,"n-1ph,14,1,1",van,320.5
1,"n-1,1,1,1",val,313.2
2,"n-4,4,4,4",val,298.2
3,"p-4,4,4,4",val,298.9
4,"n-5,5,5,5",tsac,289.2
...,...,...,...
948,n-4,6ac,258.2
949,n-2oh,6ac,306.2
950,pyr,6ac,233.2
951,n-2oh,16ac,348.7


In [7]:
i = 0
for cation in df["cation"].items():
    mapper = codes["code"] == cation[1]
    true_index = mapper[mapper].index
    smiles = codes["SMILES"][true_index].values[0]
    df["cation"][i] = smiles
    i +=1


C:\Users\01121272\AppData\Local\Temp\ipykernel_7208\955500760.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["cation"][i] = smiles
C:\Users\01121272\AppData\Local\Temp\ipykernel_7208\955500760.py:6: SettingWithCopyWarning: 
A value i

In [8]:
i = 0
for cation in df["anion"].items():
    mapper = codes["code"] == cation[1]
    true_index = mapper[mapper].index
    smiles = codes["SMILES"][true_index].values[0]
    df["anion"][i] = smiles
    i +=1


C:\Users\01121272\AppData\Local\Temp\ipykernel_7208\867885196.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["anion"][i] = smiles
C:\Users\01121272\AppData\Local\Temp\ipykernel_7208\867885196.py:6: SettingWithCopyWarning: 
A value is

In [11]:
smiles_list = []
for cation in df["cation"].items():
    for anion in df["anion"].items():
        smiles = cation[1] + "." + anion[1]
        smiles_list.append(smiles)
df["smiles"] = pd.Series(smiles_list)

In [13]:
df.to_csv("kp_database.csv")